# MLP idea

In [1]:
import os
import torch
import math
print(f"PyTorch has version {torch.__version__} with cuda {torch.version.cuda}")

PyTorch has version 2.3.1+cpu with cuda None


In [3]:
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader


In [4]:
class MLP(nn.Module):
    """
    Create a simple MLP class for future help in the creation of the GNN network.
    """
    def __init__(
        self,
        input_size,
        layer_sizes,
        output_size=None,
        output_activation=torch.nn.Identity,
        activation=torch.nn.ReLU
    ):
        super(MLP, self).__init__()
        sizes = [input_size] + layer_sizes
        if output_size is not None:
            sizes.append(output_size)
        layers = []
        for i in range(len(sizes) - 1):
            if (i < len(sizes) - 2):
                act = activation
            else:
                act = output_activation
            layers += [torch.nn.Linear(sizes[i], sizes[i + 1]), act()]
        self.model = torch.nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [63]:
# Load data
import pandas as pd
df_displ = pd.read_csv('data/displacement.csv')
df_displ = df_displ.iloc[:-1,:]
df_target = pd.read_csv("data/Data4.csv")
df_target = df_target.iloc[:,1:]


In [64]:
df_target.shape, df_displ.shape

((15000, 31), (15000, 31))

In [65]:
print(df_displ.shape, df_target.shape)

(15000, 31) (15000, 31)


In [66]:
X = df_displ.values

X_train = torch.tensor(X[:13000], dtype=torch.float32)
X_test = torch.tensor(X[13000:], dtype=torch.float32)

Y = df_target.values
Y_train = torch.tensor(Y[:13000], dtype=torch.float32)
Y_test = torch.tensor(Y[13000:], dtype=torch.float32)

In [67]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

(torch.Size([13000, 31]),
 torch.Size([13000, 31]),
 torch.Size([2000, 31]),
 torch.Size([2000, 31]))

In [68]:
# Create dataloaders
from torch.utils.data import TensorDataset
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [71]:
# Create the model 

model = MLP(X_train.shape[1], [64, 64, 64], output_size=Y_train.shape[1])
model

MLP(
  (model): Sequential(
    (0): Linear(in_features=31, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=31, bias=True)
    (7): Identity()
  )
)

In [72]:
for i, (x, y) in enumerate(train_loader):
    print(x.shape, y.shape) 
    y_hat = model(x)
    print(y_hat.shape)
    break

torch.Size([32, 31]) torch.Size([32, 31])
torch.Size([32, 31])


In [73]:
# Define the loss function and the optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Also include a scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

# Train the model

n_epochs = 1000
losses = []
loss_val = 0 
for epoch in range(n_epochs):
    model.train()

    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        loss_val += loss.item()

    losses.append(loss_val/len(train_loader))
    scheduler.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.24019259214401245


KeyboardInterrupt: 

In [42]:
# Evaluate the model
model.eval()
with torch.no_grad():
    for i, (x, y) in enumerate(test_loader):
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(f"Test Loss: {loss.item()}")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (31x14001 and 1000x64)

In [44]:
X_train.shape

torch.Size([31, 1000])